# How do I _copy_ an app from one of my other _projects_?
### Overview
Apps can be copied from one of two sources:
1. other projects you are a member of
2. public reference. 

Here we focus on copying an app from _another project_. 

### Prerequisites
 1. You need to be a member (or owner) of _at least one_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 4. You understand how to <a href="apps_listAll.ipynb" target="_blank">list</a> apps within your project (we will just use that call directly and pick one here).
 
## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The _Api_ object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify platform {cgc, sbg}
prof = 'sbpla'

config_config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_config_file)

## Copy our own App
We will first list all our projects, then list the apps within the first project, and finally copy an app between the first and second project. (Note that here there is a chance to copy also a specific version of the app, by changing the last character of the following variable to the version you want:
```python
my_apps_source.id[a_index]
```
The critical information for this POST is the **app_id**. Note, you are **not** allowed to copy the same app **and** assign the same name<sup>1</sup> more than once. If you change the name, it is ok.

To make these results very obvious, use an empty project as your TARGET\_PROJECT (e.g. your <a href=projects_makeNew.ipynb> cookbook example project</a>) or change the _name_ in the _data_ dictionary to something like 'Dept of Awesome'<sup>1</sup>

<sup>1</sup> Note that setting the **name** of an app, actually changes the **id**. We are working on fixing this inconsistency.

#### Why am I getting an Error Code 6000?
You found a known bug, we are _very sorry_ for this. It is already being worked on and should be ok soon. Known affected workflows:
* Exome Coverage QC 1.0
* RNA-Seq De Novo Assembly and Analysis - Trinity 2.0.6
* RNA-Seq De Novo Assembly - Trinity 2.0.6
* Whole Exome Sequencing GATK 2.3.9.-lite

In [ ]:
# [USER INPUT] Set project name; project (p_) and file (f_) indices here:
source_project_name = 'Keep on Smiling'
my_project_name = 'Life is Beautiful'
app_id = 'jack_digi/keep-on-smiling/TorrentMappingAlignmentProgramMap4/0'


# pull out target project
my_project = [p for p in api.projects.query(limit=100).all() \
              if p.name == my_project_name]
source_project = [p for p in api.projects.query(limit=100).all() \
              if p.name == source_project_name]

# Double-check that target project exist
if not source_project:
    print('Source project (%s) not found, check spelling' % source_project_name)
    raise KeyboardInterrupt
else:
    source_project = source_project[0]
    
if not my_project:
    print('Target project (%s) not found, check spelling' % my_project_name)
    raise KeyboardInterrupt
else:
    my_project = my_project[0]
    
my_apps = api.apps.query(project = my_project.id, limit=100)
my_app_source = [a for a in api.apps.query(project = source_project.id, limit=100) \
                 if a.id == app_id][0]

duplicate_app = [a for a in my_apps.all() if a.name == my_app_source.name]

if duplicate_app:
    print('App already exists in second project, please try another app')
else:
    print('App (%s) does not exist in Project (%s); copying now' % \
          (my_app_source.name, my_project.name))
    
    my_new_app = my_app_source.copy(project = my_project.id, name = my_app_source.name)
        
    # re-list apps in target project to verify the copy worked
    my_apps = api.apps.query(project = my_project.id, limit=100)
    my_app_names = [a.name for a in my_apps.all()]
    
    if my_app_source.name in my_app_names:
        print('Sucessfully copied one app!')
    else:
        print('Something went wrong...') 

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/copy-an-app)